# Introduction to synthetic experimentation

This notebook introduces the concept of synthetic experiments and demonstrates how to implement one using Python.

Synthetic experiments are a powerful tool for studying the relationship between experimental conditions (e.g., ratio and scatteredness in our 2AFC task) and observations (e.g., response times) in a controlled environment. They allow us to collect synthetic data that resembles real empirical data but is less complex, only as noisy as we want it to be, and free from real-world limitations such as financial and time restrictions.

Synthetic experiments use cognitive models instead of real participants. In cognitive science, this means replacing human participants with mathematical models that produce similar (but simpler) behavior. A cognitive model can be as simple as a single mathematical expression, making it much easier to evaluate our experimental designs and adaptive algorithms.

This way, the concepts and algorithms from the 'Optimizing Experimental Design' course can be implemented, tested, and validated easily before running real experiments.

In this tutorial, you will learn how to:

- Implement a cognitive model that simulates how response times depend on experimental conditions (ratio and scatteredness).
- Understand how model parameters represent individual differences between synthetic participants.
- Generate realistic data by adding measurement noise to the observations.
- Collect a dataset from multiple synthetic participants tested on different conditions.
- Fit statistical models to recover the cognitive model parameters from the data.
- Understand how sample size and noise level affect parameter recovery quality.

By the end of this tutorial, you will have a good understanding of how to implement and analyze synthetic experiments using Python.

# Experiment description

We want to examine the response times of participants in a two-alternative-forced-choice (2AFC) experiment. This experiment consists of an image which is shown for a short period of time. The image is a grid of either orange or blue tiles as shown in the picture below. 

We can control the experiment via the two factors **ratio** and **scatterdness**. 
**Ratio** determines the amount of blue vs orange tiles, where 0 means that the participant sees only orange tiles while 1 means that the amount of blue vs orange tiles is perfectly balanced. 
**Scatterdeness** determines how noisy the image appears, where 0 means that all orange tiles are placed on the left half while 1 means that the tiles are placed completely randomly.

![static/img/2afc_grid.png](static/img/2afc_grid.png)

(Image source: Trueblood J. S. et al (2021)., Urgency, Leakage, and the Relative Nature of Information Processing in
Decision-Making.)

## Import all the relevant libraries and packages

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from typing import Callable, Iterable

np.random.seed(42)

# Add the path of the project folder to the python variables
# That way python finds custom packages defined in the project folder
target_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust path as needed
if target_folder not in sys.path:
    sys.path.append(target_folder)

## Implementing a cognitive model

In this section we will implement a cognitive model that simulates how participants respond in the 2AFC experiment. This model will generate response times based on the experimental conditions (ratio and scatteredness).

The model includes **parameters** that represent individual differences between participants. For example, some participants might be generally faster or more sensitive to certain conditions than others. By varying these parameters, we can simulate different synthetic participants with different behavioral patterns.

In this section, you will learn to:
- Implement the cognitive model as a mathematical function that takes experimental conditions and participant parameters as input
- Understand what the parameters represent cognitively (e.g., sensitivity to ratio vs. scatteredness)
- Visualize how the model predicts response times across different experimental conditions
- See how changing parameters creates different response patterns

### Implement the cognitive model

The cognitive model is a mathematical function that takes the experimental conditions (ratio and scatteredness) as input and returns a predicted response time.

Our model has two parameters:
- **parameters[0]**: Controls sensitivity to the ratio (how balanced blue vs. orange tiles are)
- **parameters[1]**: Controls sensitivity to scatteredness (how randomly distributed the tiles are)

Different parameter values create different behavioral patterns, simulating individual differences between participants.

In [ ]:
def cognitive_model(ratio, scatteredness, parameters=np.ones(2,)):
    """This cognitive model predicts response times in the 2AFC task
    
    Args:
        ratio (float): The balance of blue vs orange tiles (0 to 1)
        scatteredness (float): How randomly distributed the tiles are (0 to 1)
        parameters (Iterable[float], optional): Individual participant parameters. 
            parameters[0]: sensitivity to ratio
            parameters[1]: sensitivity to scatteredness

    Returns:
        float: Predicted response time
    """
    
    # This is a bell-shaped function that can saturate
    # Response time increases with ratio (more balanced = harder decision)
    # Response time also increases with scatteredness (more scattered = harder to process)
    response_time = (1 - np.exp(-np.power(ratio, 2) / parameters[0])) + np.power(scatteredness, parameters[1])
    
    return response_time

Test your cognitive model by giving it different parameter values and check if it produces plausible response time patterns.

You can modify the `parameters` variable to see how different participants might behave. For example:
- Higher values of parameters[0] mean less sensitivity to ratio (flatter response to changes in blue/orange balance)
- Higher values of parameters[1] mean stronger effect of scatteredness on response time

This code visualizes the predicted response times across all combinations of ratio and scatteredness for one synthetic participant.

In [ ]:
# Define the experimental conditions to test
ratio = np.linspace(0, 1)
scatteredness = np.linspace(0, 1)

# Define parameters for one synthetic participant
# Try changing these values to see how behavior changes!
parameters = [0.5, 2]

# DO NOT TOUCH THE CODE FROM HERE!

# Set a sample size
sample_size = len(ratio)

# Initialize the response_time array
response_time = np.zeros((sample_size, sample_size))

# Collect the predictions from the cognitive model
ratio_mesh, scatteredness_mesh = np.meshgrid(ratio, scatteredness)
for i in range(sample_size):
    response_time[i, :] = cognitive_model(ratio_mesh[i], scatteredness_mesh[i], parameters)

# Make a surface plot to visualize the cognitive model predictions
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(ratio_mesh, scatteredness_mesh, response_time, cmap=cm.Blues)
ax.set_title('Cognitive Model: Response times across conditions')
ax.set_xlabel('Ratio')
ax.set_ylabel('Scatteredness')
ax.set_zlabel('Response time')
plt.show()

Does the pattern look plausible? 

Try different parameter configurations to get a feeling for how response times change from participant to participant. For instance:
- Try parameters = [0.5, 1] vs. [2, 1] to see how sensitivity to ratio affects the pattern
- Try parameters = [1, 0.5] vs. [1, 3] to see how sensitivity to scatteredness affects the pattern

Afterwards you can move to the next part!

## Adding measurement noise

Real experiments always have measurement noise - random variability in the observations even when the experimental conditions are identical. This noise comes from many sources: momentary lapses in attention, motor variability, random fluctuations in the environment, etc.

To make our synthetic data more realistic, we add random noise to each simulated response time. The noise is drawn from a normal distribution with mean 0, meaning it doesn't systematically bias the data up or down, just adds random variability.

The **noise level** parameter controls how much variability we add:
- noise_level = 0: Perfect, noise-free data (unrealistic)
- noise_level = 0.1: Low noise (very controlled experiment)
- noise_level = 0.5: Moderate noise (typical experiment)
- noise_level = 1.0: High noise (difficult experimental conditions)

We'll use numpy's `np.random.normal(mean, std)` function to generate random noise values.

Let's visualize what noise looks like by generating a series of random noise samples:

In [ ]:
noise_level = 0.5

# Generate 100 random noise samples
noise_sample_size = 100
noise_samples = np.random.normal(0, noise_level, noise_sample_size)
    
plt.plot(noise_samples, '.')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.title(f'Random noise samples (noise level = {noise_level})')
plt.ylabel('Noise value')
plt.xlabel('Sample number')
plt.show()

Now let's add noise to the cognitive model predictions and see how it affects the response time pattern:

In [ ]:
# Set noise level
noise_level = 0.2

# Initialize the noisy response_time array
response_time_noisy = np.zeros((sample_size, sample_size))

# Collect the observations from the cognitive model
for i in range(sample_size):
    response_time_noisy[i, :] = cognitive_model(ratio_mesh[i], scatteredness_mesh[i], parameters)
    
# Add noise to each observation
for i in range(response_time_noisy.shape[0]):
    for j in range(response_time_noisy.shape[1]):
        response_time_noisy[i, j] += np.random.normal(0, noise_level)

# Make sure no response times are negative (response times must be positive)
response_time_noisy = np.maximum(response_time_noisy, 0)

# Compare clean vs. noisy predictions
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 6))

# Clean cognitive model
ax1.plot_surface(ratio_mesh, scatteredness_mesh, response_time, cmap=cm.Blues)
ax1.set_title('Clean cognitive model (no noise)')
ax1.set_xlabel('Ratio')
ax1.set_ylabel('Scatteredness')
ax1.set_zlabel('Response time')

# Noisy observations
ax2.plot_surface(ratio_mesh, scatteredness_mesh, response_time_noisy, cmap=cm.Blues)
ax2.set_title(f'With measurement noise (level = {noise_level})')
ax2.set_xlabel('Ratio')
ax2.set_ylabel('Scatteredness')
ax2.set_zlabel('Response time')

plt.tight_layout()
plt.show()

How did the observations change? Can you still see the underlying pattern? 

Try increasing the noise level gradually (e.g., 0.1, 0.5, 1.0, 2.0) to see how the signal-to-noise ratio decreases and the pattern becomes harder to detect.

## Creating synthetic participants

We have already implemented a `SyntheticParticipant` class in the `resources.synthetic` module that combines the cognitive model and noise generation. This class represents a single synthetic participant with:
- Their own cognitive model parameters (individual differences)
- A specified noise level (measurement variability)

Each synthetic participant can then be "tested" on different experimental conditions to generate simulated response times, just like testing a real participant in the lab.

Let's import the class and create some synthetic participants with different parameters:

Let's create two synthetic participants with different parameter values and compare their predicted response times across the same experimental conditions.

Participant 1 will have parameters [1, 1] while Participant 2 will have parameters [2, 2], making them differentially sensitive to the experimental manipulations.

In [ ]:
# Import the SyntheticParticipant class
from resources.synthetic import experimental_unit as SyntheticParticipant

# Define parameters for two different participants
parameters_participant1 = [1, 1]
parameters_participant2 = [2, 2]

# Create two synthetic participants with different parameters
participant1 = SyntheticParticipant(
    cognitive_model=cognitive_model,
    parameters=parameters_participant1,
    noise_level=0,  # No noise for now, so we can see the pure parameter effect
)

participant2 = SyntheticParticipant(
    cognitive_model=cognitive_model,
    parameters=parameters_participant2,
    noise_level=0,
)

# Define the experimental conditions to test
ratio = np.linspace(0, 1)
scatteredness = np.linspace(0, 1)
ratio_mesh, scatteredness_mesh = np.meshgrid(ratio, scatteredness)
sample_size = len(ratio)

# Collect observations from participant 1
response_time_p1 = np.zeros((sample_size, sample_size))
for i in range(sample_size):
    response_time_p1[i, :] = participant1.step(ratio_mesh[i], scatteredness_mesh[i], noise=True).reshape(-1)
    
# Collect observations from participant 2  
response_time_p2 = np.zeros((sample_size, sample_size))
for i in range(sample_size):
    response_time_p2[i, :] = participant2.step(ratio_mesh[i], scatteredness_mesh[i], noise=True).reshape(-1)

# Visualize both participants side by side
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(14, 6))

ax1.plot_surface(ratio_mesh, scatteredness_mesh, response_time_p1, cmap=cm.Blues)
ax1.set_title(f'Participant 1 (params={parameters_participant1})')
ax1.set_xlabel('Ratio')
ax1.set_ylabel('Scatteredness')
ax1.set_zlabel('Response time')

ax2.plot_surface(ratio_mesh, scatteredness_mesh, response_time_p2, cmap=cm.Greens)
ax2.set_title(f'Participant 2 (params={parameters_participant2})')
ax2.set_xlabel('Ratio')
ax2.set_ylabel('Scatteredness')
ax2.set_zlabel('Response time')

plt.tight_layout()
plt.show()

## Generating a dataset from multiple synthetic participants

Now we'll generate a complete dataset by "testing" multiple synthetic participants on various experimental conditions. This simulates running a real experiment with many participants.

The dataset will have:
- **Between-participant variability**: Different participants have different parameters
- **Measurement noise**: Each observation includes random noise

We'll use a pre-defined function from `resources.synthetic` to generate the dataset efficiently.

In this section you will learn to:
- Define the study parameters (number of participants, number of conditions tested)
- Generate diverse synthetic participants with varying parameters
- Create a dataset by testing all participants on randomly sampled conditions
- Understand the structure of the resulting dataset

First, let's define the parameters for our simulated study:

In [ ]:
# Define the dataset parameters

# Number of synthetic participants to create
n_participants = 100

# Number of different conditions each participant will be tested on
n_conditions = 100

# Measurement noise level
noise_level = 0.5

Now we'll generate diverse participant parameters and experimental conditions:

In [ ]:
# Generate participant parameters
# We draw parameters from a normal distribution around 1, creating individual differences
# Parameters are constrained to be positive (can't have negative sensitivity)
parameters = np.random.normal(1, 0.5, (n_participants, 2))
parameters = np.where(parameters < 0, 0.1, parameters)  # Replace any negative values with 0.1
parameters = np.where(parameters > 2, 1.9, parameters)

# Generate experimental conditions
# We sample random combinations of ratio and scatteredness
conditions = np.random.uniform(0, 1, (n_conditions, 2))

print(f"Created {n_participants} synthetic participants")
print(f"Generated {n_conditions} experimental conditions")
print(f"Example participant parameters: {parameters[0]}")
print(f"Example condition: ratio={conditions[0, 0]:.3f}, scatteredness={conditions[0, 1]:.3f}")

Now let's create all synthetic participants and collect data from them:

In [ ]:
# Import the dataset generation function
from resources.synthetic import generate_dataset

# Create synthetic participants
synthetic_participants = []
for i in range(n_participants):
    synthetic_participants.append(
        SyntheticParticipant(
            cognitive_model=cognitive_model,
            parameters=parameters[i],
            noise_level=noise_level,
        )
    )

# Generate the dataset
# This tests all participants on all conditions and splits into train/test
dataset_train, dataset_test = generate_dataset(
    experimental_units=synthetic_participants,
    conditions=conditions,
    train_ratio=0.8,  # 80% for training, 20% for testing
)

print(f"Training dataset shape: {dataset_train.shape}")
print(f"Test dataset shape: {dataset_test.shape}")
print(f"Columns: [participant_id, ratio, scatteredness, response_time]")

Let's examine the dataset and visualize how response times vary across participants:

In [ ]:
# Show first few rows of the dataset
print("First 5 rows of training data:")
print(dataset_train[:5])
print()

# Compute participant-level averages
participant_means = []
participant_stds = []
for p_id in range(n_participants):
    p_data = dataset_train[dataset_train[:, 0] == p_id, -1]
    if len(p_data) > 0:
        participant_means.append(p_data.mean())
        participant_stds.append(p_data.std())

# Visualize between-participant variability
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot mean response time per participant
ax1.bar(range(len(participant_means)), participant_means, alpha=0.7)
ax1.set_xlabel('Participant ID')
ax1.set_ylabel('Mean response time')
ax1.set_title('Average response time by participant')

# Plot within-participant variability
ax2.bar(range(len(participant_stds)), participant_stds, alpha=0.7, color='orange')
ax2.set_xlabel('Participant ID')
ax2.set_ylabel('Std of response time')
ax2.set_title('Response time variability by participant')

plt.tight_layout()
plt.show()

You should see variability both between and within participants. The between-participant variability comes from different parameter values, while the within-participant variability comes from measurement noise.

How would you expect the variability to change if you increased the number of conditions tested or increased the noise level? Test your intuition by re-running the dataset generation with different values!

## Model fitting

In this section we'll fit statistical models to our synthetic dataset to recover the underlying cognitive model parameters. This demonstrates a key use of synthetic experiments: we know the true parameters, so we can test how well our analysis methods work.

We'll fit two types of models:
1. **Linear regression**: A simple group-level model that estimates average effects across all participants
2. **Neural network**: A more flexible model that can capture individual differences and non-linear patterns

In this section you will learn to:
- Prepare data for model fitting (the data is already split into train/test)
- Fit models to recover cognitive parameters from behavioral data
- Evaluate how well the models recover the true underlying patterns
- Understand how sample size and noise affect parameter recovery quality

### Data preparation

The dataset is already split into training and test sets by the `generate_dataset` function:
- **Training data**: Used to fit the model parameters
- **Test data**: Used to evaluate how well the model generalizes to new conditions

Let's check what we have:

In [ ]:
print(f'Number of training samples: {len(dataset_train)}')
print(f'Number of test samples: {len(dataset_test)}')
print(f'Dataset structure: [participant_id, ratio, scatteredness, response_time]')
print(f'\nExample training samples:')
print(dataset_train[:3])

### Fitting a linear regression model

We'll use a simple linear regression model from sklearn. This model estimates group-level effects - the average influence of ratio and scatteredness across all participants.

**Note**: Linear regression won't capture the non-linear pattern in our cognitive model perfectly, but it gives us a baseline to see how well a simple model performs.

In [ ]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
# We use columns 1:3 (ratio and scatteredness) to predict column 3 (response time)
# We ignore participant_id since linear regression doesn't model individual differences
model.fit(X=dataset_train[:, 1:3], y=dataset_train[:, 3])

# Extract the fitted coefficients
coef_ratio = model.coef_[0]
coef_scatteredness = model.coef_[1]

# Compare to true average parameters
true_params_mean = np.mean(parameters, axis=0)
print('Parameter Recovery:')
print(f'True average parameters: ratio_sensitivity={true_params_mean[0]:.3f}, scatteredness_sensitivity={true_params_mean[1]:.3f}')
print(f'Linear model coefficients: ratio={coef_ratio:.3f}, scatteredness={coef_scatteredness:.3f}')
print(f'\nNote: Linear regression coefficients are not directly comparable to cognitive model parameters')
print(f'because our cognitive model is non-linear. But they give us a sense of the trends.')

Now let's evaluate the linear model on the test data:

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Use the fitted model to predict test observations
predictions = model.predict(dataset_test[:, 1:3])

# Compute prediction metrics
mse = mean_squared_error(dataset_test[:, 3], predictions)
r2 = r2_score(dataset_test[:, 3], predictions)

print(f'Test set performance:')
print(f'Mean Squared Error: {mse:.4f}')
print(f'R² Score: {r2:.4f}')
print(f'\nOriginal mean response time: {np.mean(dataset_test[:, 3]):.3f}')
print(f'Predicted mean response time: {np.mean(predictions):.3f}')

# Visualize the fitted model's predictions
ratio = np.linspace(0, 1, 50)
scatteredness = np.linspace(0, 1, 50)
ratio_mesh, scatteredness_mesh = np.meshgrid(ratio, scatteredness)
sample_size = len(ratio)

# Get model predictions
response_time_predicted = np.zeros((sample_size, sample_size))
for i in range(sample_size):
    conditions = np.stack((ratio_mesh[i], scatteredness_mesh[i]), axis=-1)
    response_time_predicted[i, :] = model.predict(conditions)

# Plot the linear model's predictions
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(ratio_mesh, scatteredness_mesh, response_time_predicted, cmap=cm.Reds, alpha=0.7)
ax.set_title('Linear Model Predictions')
ax.set_xlabel('Ratio')
ax.set_ylabel('Scatteredness')
ax.set_zlabel('Response time')
plt.show()

The R² score tells you what proportion of variance in response times the model explains (1.0 = perfect, 0.0 = no better than guessing the mean).

The linear model approximates the overall trend but can't capture the non-linear pattern perfectly. Try adjusting the number of participants, conditions, or noise level to see how it affects model performance!

### Fitting a neural network

Now let's fit a more flexible neural network model. Unlike linear regression, the neural network can:
- Learn non-linear patterns in the data
- Model individual differences between participants (by using participant_id as input)

We'll use a pre-defined feed-forward network from `resources.regressors`.

In [ ]:
from resources.regressors import FFN, FFNRegressor

# Train a neural network model
# The network takes [participant_id, ratio, scatteredness] as input
model_ffn = FFNRegressor(FFN(n_participants, 2), max_epochs=100, lr=0.1)
model_ffn.fit(dataset_train[:, 0:3], dataset_train[:, 3:4].astype(np.float32))

# Get predictions on the test data
predictions_ffn = model_ffn.predict(dataset_test[:, 0:3])
mse_ffn = mean_squared_error(dataset_test[:, 3], predictions_ffn)
r2_ffn = r2_score(dataset_test[:, 3], predictions_ffn)

print(f"Neural Network Test Performance:")
print(f"Mean Squared Error: {mse_ffn:.4f}")
print(f"R² Score: {r2_ffn:.4f}")
print(f"\nComparison to Linear Regression:")
print(f"Linear MSE: {mse:.4f} vs Neural Net MSE: {mse_ffn:.4f}")
print(f"Linear R²: {r2:.4f} vs Neural Net R²: {r2_ffn:.4f}")

How does the neural network compare to linear regression? 

The neural network should perform better (lower MSE, higher R²) because it can learn the non-linear cognitive model pattern and individual differences. However, it needs more data to train effectively.

Let's visualize how well the neural network recovers individual participant behavior:

In [ ]:
# Visualize neural network predictions for a specific participant
participant_id = 1

# Define conditions to test
ratio = np.linspace(0, 1, 50)
scatteredness = np.linspace(0, 1, 50)
ratio_mesh, scatteredness_mesh = np.meshgrid(ratio, scatteredness)
sample_size = len(ratio)

# Get neural network predictions for this participant
response_time_nn = np.zeros((sample_size, sample_size))
for i in range(sample_size):
    conditions = np.stack((ratio_mesh[i], scatteredness_mesh[i]), axis=-1)
    participant_id_array = np.full((conditions.shape[0], 1), participant_id)
    X = np.concatenate((participant_id_array, conditions), axis=-1)
    response_time_nn[i, :] = model_ffn.predict(X).reshape(-1)

# Get true response times from the cognitive model
response_time_true = np.zeros((sample_size, sample_size))
for i in range(sample_size):
    response_time_true[i, :] = synthetic_participants[participant_id].step(ratio_mesh[i], scatteredness_mesh[i], noise=False).reshape(-1)

# Plot comparison
fig, (ax1) = plt.subplots(1, 1, subplot_kw={"projection": "3d"}, figsize=(14, 6))

# True cognitive model
ax1.plot_surface(ratio_mesh, scatteredness_mesh, response_time_true, cmap=cm.Blues, alpha=0.7)
ax1.plot_surface(ratio_mesh, scatteredness_mesh, response_time_nn, cmap=cm.Reds, alpha=0.7)
ax1.set_title(f'True Cognitive Model vs Neural Network Prediction (Participant {participant_id})')
ax1.set_xlabel('Ratio')
ax1.set_ylabel('Scatteredness')
ax1.set_zlabel('Response time')

plt.tight_layout()
plt.show()

print(f"You can change 'participant_id' to visualize different participants (0 to {n_participants-1})")

## Congratulations!

You've completed the first tutorial on synthetic experimentation!

You should now have a good understanding of:
- How to implement a cognitive model to simulate participant behavior
- How model parameters represent individual differences between participants
- How measurement noise affects experimental data
- How to generate synthetic datasets from multiple participants
- How to fit and evaluate statistical models to recover cognitive parameters
- How sample size and noise level impact parameter recovery quality

In the next tutorials, you'll learn how to optimize experimental designs using these synthetic experiments to test different experimental strategies before running real studies!